In [1]:
from datetime import datetime
import json

In [2]:
import anvil.server

anvil.server.connect("client_KYOM4YFDIE4TMEO3UAOTEPRC-55C7JZ62MGB2UXA6")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Published" as CLIENT


In [3]:
from anvil.tables import app_tables

questions = app_tables.questions.search()

In [4]:
from dotenv import load_dotenv
from groq import Groq
import json_repair
import os

load_dotenv()

key_groq = os.environ["GROQ_API_KEY"]
client = Groq(api_key=key_groq)

In [5]:
json_schema = """
  {
    "topic_description": {
      "type": "string",
      "description": "A sentence describing the sub-topic to which the question belongs. That means this sentence should specify in a granular level what specific sub-topic the question belongs to. It should be abstract in a way that other questions could be put in this description too. Use between 5 and 10 words."
    },
    "level": {
      "type": "string",
      "description": "The difficulty level of the question. It should be only one of the following options: 'beginner', 'intermediate', 'advanced'."
    },
    "question": {
      "type": "string",
      "description": "The actual question text. It should be a question of type TRUE or FALSE. It means that the questions should be an assertion that could be answered with TRUE or FALSE."
    },
    "answer_correct": {
      "type": "string",
      "description": "The correct answer to the question. It should be only one of the following options: TRUE or FALSE"
    },
    "explanation": {
      "type": "string",
      "description": "An explanation or solution to the question."
    }
  }
"""

In [6]:
prompt_question_generator = """
    TASK CONTEXT:
    I am studying machine learning and I need to practice some questions on various topics.
    
    TASK DESCRIPTION:
    I will provide you with a list of topics, and I would like you to generate a list of TRUE or FALSE questions.
    These questions should be interesting, creative, challenging and thought-provoking. 
    Each question should be in the form of a statement that could be either TRUE or FALSE.
    Feel free to be imaginative and attempt to confuse the student by blending related concepts or similar words.
    I will provide the topics in the DOMAIN KNOWLEDGE section.
    The questions should pertain to these topics, and you can use this knowledge as a foundation to create questions that delve deeper into the subject matter.
    
    ADDITIONAL TASK DESCRIPTION:
    {additional_task_description}
    
    TASK REQUIREMENTS:
    Please refrain from creating questions that require mathematical calculations, but you may create questions with mathematical formulas.
    You SHOULD use LATEX to write mathematical formulas and code, but you should use the Katex flavor.
    Also you should put $$ in the beggining of the katex code and $$ at the end of the code. This is necessary because the interpreter needs it.
    
    TASK DETAILS:
    You should create {quantity} questions of level {level}.
    
    DOMAIN KNOWLEDGE:
    {domain_knowledge}
    
    FORMAT OUTPUT INSTRUCTIONS:
    It should be formatted in list of JSON objects as described below.
    {json_schema}
"""

In [7]:
parameters = {
    "additional_task_description": "",
    "quantity": "1",
    "level": "easy",
    "domain_knowledge": "data structures and algorithms",
}

In [8]:
prompt_question_generator_formatted = prompt_question_generator.format(
    additional_task_description=parameters["additional_task_description"],
    quantity=parameters["quantity"],
    level=parameters["level"],
    domain_knowledge=parameters["domain_knowledge"],
    json_schema=json_schema,
)

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "user", "content": prompt_question_generator_formatted}],
    temperature=1,
    max_tokens=2420,
    top_p=1,
    stream=False,
    response_format={"type": "json_object"},
    stop=None,
)

completion = response.choices[0].message
questions = json_repair.loads(completion.content)

In [9]:
if not isinstance(questions, list):
        questions = [questions]

for question in questions:
    print(json.dumps(question, indent=4))
    # add to the database
    # app_tables.questions.add_row(
    #     created_at=datetime.now(),
    #     title=parameters["title"],
    #     topic_description=question["topic_description"],
    #     level=question["level"],
    #     question=question["question"],
    #     type="true_or_false",
    #     answer_correct=question["answer_correct"],
    #     answers=None,
    #     explanation=question["explanation"],
    #     user=anvil.users.get_user(),
    # )

{
    "topic_description": "Data Structures: Sorting Algorithms",
    "level": "easy",
    "question": "A hash function in quicksort automatically sorts the array",
    "answer_correct": "FALSE",
    "explanation": "Hash functions in quicksort are used to partition the array, not to sort it. The actual sorting is done through the recursive partitioning process."
}
